In [1]:
import pandas as pd
from pulp import *

In [2]:
df = pd.read_excel('assignment_ps.xlsx')
df

,Unnamed: 0,demand,fixed cost,var,storage cost,Capacity
0,period 1,100,2000,40,1,5000
1,period 2,4000,2000,50,1,6000
2,period 3,2500,2000,80,1,4000
3,period 4,4000,2000,60,1,12000
4,period 5,6000,2000,50,1,4000
5,period 6,100,2000,60,1,4000
6,period 7,250,3000,20,1,8000
7,period 8,5000,3000,40,1,4000
8,period 9,6000,3000,90,1,4000
9,period 10,4000,3000,30,1,7000


In [3]:
df = df.rename(columns = {'Unnamed: 0' : 'period', 'Capacity' : 'capacity'})
df['t'] = list(range(1, 13))
df = df.set_index('t')
df

,period,demand,fixed cost,var,storage cost,capacity
t,,,,,,
1,period 1,100,2000,40,1,5000
2,period 2,4000,2000,50,1,6000
3,period 3,2500,2000,80,1,4000
4,period 4,4000,2000,60,1,12000
5,period 5,6000,2000,50,1,4000
6,period 6,100,2000,60,1,4000
7,period 7,250,3000,20,1,8000
8,period 8,5000,3000,40,1,4000
9,period 9,6000,3000,90,1,4000


In [4]:
inventory = LpVariable.dicts('inventory', indices = list(range(1, 13)), lowBound = 0, upBound = None, cat = 'Integer')
inventory[0] = 200

production = LpVariable.dicts('production quantity', indices = list(range(1, 13)), lowBound = 0, upBound = None, cat = 'Integer')

binary = LpVariable.dicts('binary', indices = list(range(1, 13)), lowBound = 0, upBound = None, cat = 'Binary')

time_period = list(range(1,13))

# create model
model = LpProblem('production_scheduling', LpMinimize)

# objective function
model += lpSum([binary[t] * df['fixed cost'][t] + production[t] * df['var'][t] + inventory[t] * df['storage cost'][t] for t in time_period])

# constraint
for t in time_period:
    model += production[t] - inventory[t] + inventory[t-1] >= df['demand'][t]
    model += production[t] <= binary[t] * df['capacity'][t]

# solve
model.solve()

# check result
for i in production:
    print(production[i], production[i].varValue)

value(model.objective)

production_quantity_1 5000.0
production_quantity_2 6000.0
production_quantity_3 0.0
production_quantity_4 1400.0
production_quantity_5 4000.0
production_quantity_6 1250.0
production_quantity_7 8000.0
production_quantity_8 4000.0
production_quantity_9 0.0
production_quantity_10 7000.0
production_quantity_11 4000.0
production_quantity_12 0.0


1542550.0

In [5]:
result = pd.DataFrame({'demand' : df['demand'], 'production' : [production[i].varValue for i in production], 'inventory' : [inventory[i].varValue for i in range(1, 13)], 'binary' : [binary[i].varValue for i in binary]})
result

,demand,production,inventory,binary
t,,,,
1,100,5000.0,5100.0,1.0
2,4000,6000.0,7100.0,1.0
3,2500,0.0,4600.0,0.0
4,4000,1400.0,2000.0,1.0
5,6000,4000.0,0.0,1.0
6,100,1250.0,1150.0,1.0
7,250,8000.0,8900.0,1.0
8,5000,4000.0,7900.0,1.0
9,6000,0.0,1900.0,0.0
